In [1]:
from main import SkinCareAdvisor

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import cv2
import matplotlib.pyplot as plt
import time
import json
def printImage(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.show()

In [7]:
#major integration
import dlib
from wide_resnet import WideResNet
import os
from cntk import load_model, combine
import config
import pandas as pd
import pickle

#prod_cat
PRODUCT_CATALOGUE_CSV_PATH = config.PRODUCT_CATALOGUE_CSV_PATH
prod_cat = pd.read_csv(PRODUCT_CATALOGUE_CSV_PATH)

#facial_landmark
F_L_PREDICTOR_PATH = config.F_L_PREDICTOR_PATH
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(F_L_PREDICTOR_PATH)

#age_gender
A_G_MODEL_PATH = config.A_G_MODEL_PATH
ag_model = WideResNet(64)()
ag_model.load_weights(A_G_MODEL_PATH)

#dark_eye
D_E_MODEL_PATH = config.D_E_MODEL_PATH
with open(D_E_MODEL_PATH, 'rb') as f:
    de_model = pickle.load(f)

#acne
ACNE_PRETRAINED_MODEL_NAME = config.ACNE_PRETRAINED_MODEL_NAME
ACNE_PRETRAINED_MODEL_PATH = config.ACNE_PRETRAINED_MODEL_PATH
ACNE_PRETRAINED_NODE_NAME = config.ACNE_PRETRAINED_NODE_NAME
ACNE_REGRESSION_MODEL_PATH = config.ACNE_REGRESSION_MODEL_PATH
model_file  = os.path.join(ACNE_PRETRAINED_MODEL_PATH, ACNE_PRETRAINED_MODEL_NAME)
ac_loaded_model  = load_model(model_file)
node_in_graph = ac_loaded_model.find_by_name(ACNE_PRETRAINED_NODE_NAME)
ac_output_nodes  = combine([node_in_graph.owner])
read_model = pd.read_pickle(ACNE_REGRESSION_MODEL_PATH)
regression_model = read_model['model'][0]
ac_train_regression = pickle.loads(regression_model)


c:\users\denis\anaconda3\envs\fyp\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\denis\anaconda3\envs\fyp\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\denis\anaconda3\envs\fyp\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neural_network.multilayer_perceptron module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from s

In [8]:
q = {
    'age' : None,
    'skin_type': None,
    'allergies': None,
    'price': None,
    'concerns': None,
    'preferences': None
}
image = cv2.imread("./test_images/image5.jpg")
start_time = time.time()
sca = SkinCareAdvisor(q, image, detector, predictor, prod_cat, ag_model, de_model, ac_loaded_model, ac_output_nodes, ac_train_regression)
print("Time full pipeline", time.time() - start_time, "second(s)")

Time full pipeline 3.1656198501586914 second(s)


In [9]:
sca.full_recommendations

'{"0": {"title": "Glycolic Acid 7% Toning Solution", "issue": "Cleanser", "image": {"uri": "https://www.sephora.com/productimages/sku/s2031508-main-zoom.jpg"}, "price": 8.699999999999998, "rating": 4.5, "likes": 172100, "description": "An exfoliating toning solution with seven percent glycolic acid, amino acids, aloe vera, ginseng, and tasmanian pepperberry."}, "1": {"title": "Protini\\u2122 Polypeptide Moisturizer", "issue": "Moisturizer", "image": {"uri": "https://www.sephora.com/productimages/sku/s2025633-main-zoom.jpg"}, "price": 68.0, "rating": 4.0, "likes": 218900, "description": "A protein moisturizer that combines signal peptides, growth factors, amino acids, and pygmy waterlily to improve the look of skin\\u2019s tone, texture, and firmness."}, "2": {"title": "Truth Serum\\u00ae", "issue": "Treatment", "image": {"uri": "https://www.sephora.com/productimages/sku/s1910470-main-zoom.jpg"}, "price": 50.0, "rating": 4.2, "likes": 202200, "description": "A powerful antiaging serum f